Question-1:

Analysis-2:

Let's take the crtical words as 'bankrupt','fraud','deal','agreement'
To fetch the count of these crtitcal words in the sent emails of all persons by each year

In [27]:
#To fetch all the sent files of each person
import os
from glob import glob
import glob

cwd_path = os.getcwd() 
root_dir_path=os.path.abspath(os.path.join(cwd_path, os.pardir)) 
root=root_dir_path.replace("\\", "/")

pattern = "*"

all_sent_mails=[]

sent_start_dir1=root+'/data/enron/maildir/'

import os
test_directory = sent_start_dir1

for child in os.listdir(test_directory):
    test_path = os.path.join(test_directory, child)
    if os.path.isdir(test_path):
        #print(test_path) 
        #t=test_path.find('/',0)# All persons folders
        #print(t)
        #if test_path in path:
        if test_path+'/sent':
            new_path=test_path+'/sent'
            mails=glob.glob(new_path+'/*')
            all_sent_mails+=mails
            
print('Total number of sent mails from all persons: ',len(all_sent_mails))
        

Total number of sent mails from all persons:  57653


In [28]:
# To check for critical words in each mail;
#if present add it a dictionary corresponding to the year in which the mail had been sent

import string
import nltk
import glob
import re
import string
import glob
from email.parser import Parser
import email

year_section_dictionary={} #To store year-person-mailcount

for filename in all_sent_mails:

    with open(filename,'r',encoding='utf-8',errors='ignore') as file:
        email = Parser().parsestr(file.read())
        year=str(email['Date'][11:15]) # Year in which email was sent
        if year==' 200': year=year+'0'
        wnl = nltk.WordNetLemmatizer()
        sent_words=[]
        content = email.get_payload()
        end = content.find('From',0)
        final_content=content[0:end]
        tokens=nltk.word_tokenize(final_content)
        for word in tokens:
            word=word.lower() #Converting all words to lower case 
            table = str.maketrans("", "", string.punctuation) #To remove punctuations
            word = word.translate(table)
            if word: #To remove blank strings which are formed if there is a punctuation as a word
                            #(because of space spereation in a line)
                if not word.isdigit(): # To remove numbers
                    lemmatized_word=wnl.lemmatize(word) #Lemmatizing the words
                    
                    #If word is one of these common words do not add to list
                    if not lemmatized_word in ['of','the','a','an','for','thanks','may','let','email','sr','do','thru','new','know','regard','more','ect','mr','time','cc','me','he','i','my','can','any','could','to','on','would','u','wa','and','your','yours','with','that','this','ha','http','no','at','from','all','not','in','you','is','s','will','be','it','we','have','are','or','our','am','pm','if','by','please']: #To ignore common words
                        sent_words.append(lemmatized_word)

        critical_words_list_in_file=[]
        for word in sent_words:
            #If the word is one of the 4 critical words, add to the list 
            if word in ['bankrupt','fraud','deal','agreement']: #These  4 are the critical words considered for analysis
                critical_words_list_in_file.append(word)
                #print(len(critical_words_list_in_file))
        
                        
        #Creating a dictionary of year-critical_word-count
        
        if year in year_section_dictionary: #If year is present in dictionary
            length=len(year_section_dictionary[year]) #Fetch the length of the list
            if length==0: #If list is empty append a dictionary
                 year_section_dictionary[year].append({word:0})
            else: #If list is not empty
                for i in range(length):
                    for word in critical_words_list_in_file:
                    
                        if word in year_section_dictionary[year][i]: #Check for a section name in all entries of the list
                            year_section_dictionary[year][i][word]+=1 #If present, increment the number
                        
        else: #If year is not present in dictionary
            year_section_dictionary[year]=[]
            year_section_dictionary[year].append({'bankrupt':0})
            year_section_dictionary[year].append({'fraud':0})
            year_section_dictionary[year].append({'deal':0})
            year_section_dictionary[year].append({'agreement':0})
            length=len(year_section_dictionary[year]) 
            for i in range(length):
                for word in critical_words_list_in_file:
                    if word in year_section_dictionary[year][i]: #Check for a section name 
                        year_section_dictionary[year][i][word]+=1
    
                
print('Total number of years in the dictionary: ',len(year_section_dictionary))

7


In [29]:
#Writing the output of the analysis i.e. Yearwise count of critical word to excel

import xlsxwriter
import os

cwd = os.getcwd() #To access the path of current working directory
path=cwd+'/ana_2'

# Create an excel workbook if not present and add a worksheet.
if not os.path.exists(path+'/'+'Analysis_2_Critcal_Words_Yearwise.xlsx'):
    workbook = xlsxwriter.Workbook(path+'/'+'Analysis_2_Critcal_Words_Yearwise.xlsx')

    #Adding worksheet to the workbook
    worksheet = workbook.add_worksheet('Critcal_Words_Yearwise_Data')

    #Adding headers(title) to the columns
    worksheet.write(0, 0, 'Year')
    worksheet.write(0, 1, 'Critical Word')
    worksheet.write(0, 2, 'Total Number of Words')

    # Setting rows and columns indexes
    row = 1
    col = 0

    dict_length=len(year_section_dictionary)

    for key in year_section_dictionary.keys():
        worksheet.write(row, col, key)#Year
    
        section_length=len(year_section_dictionary[key])
        for i in range(section_length):
            for sec_key,value in year_section_dictionary[key][i].items():
                worksheet.write(row, col+1, sec_key) #Critical word
                worksheet.write(row, col+2, value) #Count of critical word
            row+=1
    workbook.close()
    print('Output Data is written to file.')
    
else:
    print('The file already exists.')

Output Data is written to file.
